# Assignment 2: Retrieval-Augmented Question Answering with LLMs

In this assignment, we will use a large language model (LLM) in a retrieval-augmented setup to answer questions from the Natural Questions dataset. We will evaluate the performance of the LLM using different prompting strategies and compare the results. The steps involved are as follows:

1. **Evaluate an LLM on Natural Questions without context.**
2. **Evaluate an LLM on Natural Questions with provided context.**
3. **Set up a retriever to find relevant passages.**
4. **Evaluate the LLM using retrieved context instead of provided context.**


In [1]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

nq_data = pd.read_csv('nq_simplified.val.tsv', sep='\t', header=None, names=['question', 'answer', 'gold_context'], quoting=3)

nq_data.head()

,question,answer,gold_context
0,what purpose did seasonal monsoon winds have o...,enabled European empire expansion into the Ame...,The westerlies (blue arrows) and trade winds (...
1,who got the first nobel prize in physics,"Wilhelm Conrad Röntgen, of Germany",The award is presented in Stockholm at an annu...
2,when is the next deadpool movie being released,"May 18, 2018","Though the original creative team of Reynolds,..."
3,where did the idea of fortnite come from,as a cross between Minecraft and Left 4 Dead,"Fortnite is set in contemporary Earth, where t..."
4,which mode is used for short wave broadcast se...,MFSK Olivia,"All one needs is a pair of transceivers, each ..."


In [2]:
def rouge1(gold, predicted):
    assert(len(gold) == len(predicted))
    n_p = 0
    n_g = 0
    n_c = 0
    for g, p in zip(gold, predicted):
        g = set(cleanup(g).strip().split())
        p = set(cleanup(p).strip().split())
        n_g += len(g)
        n_p += len(p)
        n_c += len(p.intersection(g))
    pr = n_c / n_p
    re = n_c / n_g
    if pr > 0 and re > 0:
        f1 = 2*pr*re/(pr + re)
    else:
        f1 = 0.0
    return pr, re, f1

def cleanup(text):
    text = text.replace(',', ' ')
    text = text.replace('.', ' ')
    return text

## Step 1: Evaluating an LLM on Natural Questions

First, we will find an LLM on huginface hub that is small and fast to fitt on our system. 

In [3]:
"""
Load the model and tokenizer
 "mistralai/Mixtral-8x7B-Instruct-v0.1" "microsoft/Phi-3-mini-4k-instruct" "mtgv/MobileLLaMA-1.4B-Base" "TheBloke/Mistral-7B-Instruct-v0.2-AWQ" 
"TheBloke/Llama-2-7B-Chat-AWQ"

"""
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    device_map="cuda:0")


Lets create a simple pipline for QA and use a small subset of the dataset to get the predicted answers and evaluate the model using ROUGE-1 scores.

In [4]:
%%time
# Create a pipeline for question answering
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, 
                       pad_token_id=tokenizer.eos_token_id, max_new_tokens=400,)

# Define a function to get answers from the model
def get_answers(questions, prompt, max_new_tokens):
    answers = []
    for question in questions:
        p = prompt.format(question) 
        response = qa_pipeline(p,max_new_tokens=max_new_tokens)
        answers.append(response[0]['generated_text'])
    return answers

# Use a small subset of the dataset for testing
prompts = [ 
    "{}",
    '[INST] {} [/INST]',
    '[INST] {} [/INST] Answer:',
    '<s>[INST] You are a question answering system. Please give a short answer to the following user question.[/INST] Question: {}',
    '<s>Question: {} [INST]Generate a short-form answer of either one word or one sentence to the question.[/INST]',
          ]
subset = nq_data.sample(100)
questions = subset['question'].tolist()
gold_answers = subset['answer'].tolist()
results=[]
for prompt in prompts:
    predicted_answers = get_answers(questions, prompt,max_new_tokens=50)
    pr, re, f1 = rouge1(gold_answers, predicted_answers)
    result = [prompt, pr, re, f1, "without context", 100]
    results.append(result)
result = pd.DataFrame(results, columns=['Prompt', 'Rouge-1 Precision', 'Rouge-1 Recall', 'Rouge-1 F1', 'Method', 'Sample Size'])
display(result)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,Prompt,Rouge-1 Precision,Rouge-1 Recall,Rouge-1 F1,Method,Sample Size
0,{},0.046677,0.234127,0.077836,without context,100
1,[INST] {} [/INST],0.047983,0.273810,0.081657,without context,100
2,[INST] {} [/INST] Answer:,0.052021,0.293651,0.088385,without context,100
3,<s>[INST] You are a question answering system....,0.054459,0.295635,0.091975,without context,100
4,<s>Question: {} [INST]Generate a short-form an...,0.066970,0.204365,0.100881,without context,100


CPU times: user 10min 24s, sys: 265 ms, total: 10min 24s
Wall time: 10min 24s


## Step 2: An Idealized Retrieval-Augmented LLM

Now, we will include the `gold_context` in our prompts and evaluate the model again. This will help us understand the upper limits of the system.


In [14]:
%%time
# Define a function to get answers from the model with context
def get_answers_with_context(questions, contexts, prompt, max_new_tokens):
    answers = []
    for question, context in zip(questions, contexts):
        p = prompt.format(context,question) 
        response = qa_pipeline(p, max_new_tokens=max_new_tokens)
        answers.append(response[0]['generated_text'])
    return answers

contexts = subset['gold_context'].tolist()

prompts = [ 
    'Context:{}, Question: {}, Answer:',
    '[INST] Context:{}\n Question:{}[/INST]',
    '[INST] Context:{}\n Question:{}[/INST] Answer:',
    '<s>Context:{}\n [INST] You are a question answering system. Based on the above context, generate a short answer to the following question.[/INST]\n Question: {}',
    '<s>Context:{}\n Question:{} [INST] Using the information provided in the context, generate a short-form answer of either one word or one sentence to the question. [/INST]',
          ]
results=[]
for prompt in prompts:
    predicted_answers_with_context = get_answers_with_context(questions, contexts, prompt, max_new_tokens=50)
    pr_context, re_context, f1_context = rouge1(gold_answers, predicted_answers_with_context)
    answer = [prompt, pr_context, re_context, f1_context, "with context", 100]
    results.append(answer)
result_2 = pd.DataFrame(results, columns=['Prompt', 'Rouge-1 Precision', 'Rouge-1 Recall', 'Rouge-1 F1', 'Method', 'Sample Size'])
result = pd.concat([result, result_2], ignore_index=True)
result

CPU times: user 10min 7s, sys: 4.54 s, total: 10min 12s
Wall time: 10min 12s


,Prompt,Rouge-1 Precision,Rouge-1 Recall,Rouge-1 F1,Method,Sample Size
0,{},0.046677,0.234127,0.077836,without context,100
1,[INST] {} [/INST],0.047983,0.273810,0.081657,without context,100
2,[INST] {} [/INST] Answer:,0.052021,0.293651,0.088385,without context,100
3,<s>[INST] You are a question answering system....,0.054459,0.295635,0.091975,without context,100
4,<s>Question: {} [INST]Generate a short-form an...,0.066970,0.204365,0.100881,without context,100
5,"Context:{}, Question: {}, Answer:",0.130324,0.607143,0.214586,with context,100
6,[INST] Context:{}\n Question:{}[/INST],0.114516,0.593254,0.191974,with context,100
7,[INST] Context:{}\n Question:{}[/INST] Answer:,0.115204,0.583333,0.192408,with context,100
8,<s>Context:{}\n [INST] You are a question answ...,0.121236,0.615079,0.202548,with context,100
9,<s>Context:{}\n Question:{} [INST] Using the i...,0.178444,0.496032,0.262467,with context,100


## Step 3: Setting Up the Retriever

In this step, we will set up a retriever to find relevant passages. We will use the SentenceTransformers model to create embeddings for the passages and set up a FAISS index for efficient retrieval.


In [7]:
#!pip install faiss-cpu sentence_transformers

In [15]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

with open('passages.txt', 'r', encoding='utf-8') as file:
    passages = file.readlines()

embedded_passages = retriever_model.encode(passages, convert_to_tensor=True)

index = faiss.IndexFlatL2(embedded_passages.shape[1])
index.add(embedded_passages.cpu())


In [16]:
def retrieve_best_passage(question):
    question_embedding = retriever_model.encode([question], convert_to_tensor=True)
    _, ix = index.search(question_embedding.cpu(), 1)
    return passages[ix[0][0]]

question = "Where did the first African American air force unit train?"
best_passage = retrieve_best_passage(question)
print(f"Best Passage: {best_passage}")


Best Passage: Black Americans were not permitted to fly for the U.S. armed services prior to 1940. The Air Corps at that time, which had never had a single black member, was part of an army that possessed exactly two black Regular line officers at the beginning of World War II: Brigadier Generals Benjamin O. Davis, Sr. and Benjamin O. Davis, Jr. The first Civilian Pilot Training Program (CPTP) students completed their instruction in May 1940. The creation of an all-black pursuit squadron resulted from pressure by civil rights organizations and the black press who pushed for the establishment of a unit at Tuskegee, an



## Step 4: Putting the Pieces Together

Finally, we will retrieve passages for each question and evaluate the model using these passages instead of the provided `gold_context`.

In [17]:
def get_answers_with_retrieved_context(questions, prompt, max_new_tokens):
    answers = []
    for question in questions:
        context = retrieve_best_passage(question)
        prompt = f"<s> Context: {context}\n Question: {question} [INST] Using the information provided in the context, generate a short-form answer of either one word or one sentence to the question. [/INST]"
        response = qa_pipeline(prompt, max_new_tokens=max_new_tokens)
        answers.append(response[0]['generated_text'])
    return answers
prompts = [ 
    'Context:{}, Question: {}, Answer:',
    '[INST] Context:{}\n Question:{}[/INST]',
    '[INST] Context:{}\n Question:{}[/INST] Answer:',
    '<s>Context:{}\n [INST] You are a question answering system. Based on the above context, generate a short answer to the following question.[/INST]\n Question: {}',
    '<s>Context:{}\n Question:{} [INST] Using the information provided in the context, generate a short-form answer of either one word or one sentence to the question. [/INST]',
    '<s> Context: {}\n Question: {} [INST] Select the passage from the provided context that best answers the question in either one word or a sentence. [/INST]',
          ]
results=[]
for prompt in prompts:
    predicted_answers_retrieved_context = get_answers_with_retrieved_context(questions,prompt, max_new_tokens=50)
    pr_retrieved, re_retrieved, f1_retrieved = rouge1(gold_answers, predicted_answers_retrieved_context)
    answer = [prompt, pr_retrieved, re_retrieved, f1_retrieved, "Retriever", 100]
    results.append(answer)
result_3 = pd.DataFrame(results, columns=['Prompt', 'Rouge-1 Precision', 'Rouge-1 Recall', 'Rouge-1 F1', 'Method', 'Sample Size'])
result = pd.concat([result, result_3], ignore_index=True)
result

,Prompt,Rouge-1 Precision,Rouge-1 Recall,Rouge-1 F1,Method,Sample Size
0,{},0.046677,0.234127,0.077836,without context,100
1,[INST] {} [/INST],0.047983,0.273810,0.081657,without context,100
2,[INST] {} [/INST] Answer:,0.052021,0.293651,0.088385,without context,100
3,<s>[INST] You are a question answering system....,0.054459,0.295635,0.091975,without context,100
4,<s>Question: {} [INST]Generate a short-form an...,0.066970,0.204365,0.100881,without context,100
5,"Context:{}, Question: {}, Answer:",0.130324,0.607143,0.214586,with context,100
6,[INST] Context:{}\n Question:{}[/INST],0.114516,0.593254,0.191974,with context,100
7,[INST] Context:{}\n Question:{}[/INST] Answer:,0.115204,0.583333,0.192408,with context,100
8,<s>Context:{}\n [INST] You are a question answ...,0.121236,0.615079,0.202548,with context,100
9,<s>Context:{}\n Question:{} [INST] Using the i...,0.178444,0.496032,0.262467,with context,100


## Summary

In this notebook, we evaluated the performance of an LLM on the Natural Questions dataset using different prompting strategies. We observed the following:

1. **Without Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

2. **With Provided Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

3. **With Retrieved Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

Including context, whether provided or retrieved, ...
